In [1]:
# Embedding based Approach
from utils import *

In [2]:
import numpy as np

In [5]:
## SAMPLE TESTING
approach_dir = "artifacts_RAG/"
q_text = "photo finish Professional airbrush makeup"
q_img  = "https://temptupro.com/cdn/shop/products/s-one-essential-airbrush-kit-hero_2.jpg?v=1743181132&width=1780"

recs_text, scores_text = unified_query(input_text=q_text,input_image_path=None,save_dir=approach_dir,k=5)
print("Text-only recommendations:")
print(recs_text[["product_title","product_image_url"]], scores_text)

recs_img, scores_img = unified_query(input_text=None,input_image_path=q_img,save_dir=approach_dir,k=5)
print("Image-only recommendations:")
print(recs_img[["product_title","product_image_url"]], scores_img)

recs_both, scores_both = unified_query(input_text=q_text,input_image_path=q_img,save_dir=approach_dir,k=5)
print("Text+Image recommendations:")
print(recs_both[["product_title","product_image_url"]], scores_both)

Text-only recommendations:
                                       product_title  \
0  Photo Finish Professional Airbrush Cosmetic Ma...   
1   Color On Professional Eye Shadow Smokey Classics   
2                      Waterproof Long Lasting Color   
3  Fashion Fair Oil Free Perfect FinishÂ Cream-to...   
4  Eye makeup-mascara,essence mascar,Mascara to B...   

                                   product_image_url  
0  https://m.media-amazon.com/images/I/61+MdJC0T1...  
1  https://m.media-amazon.com/images/I/21+G4x+jUa...  
2  https://m.media-amazon.com/images/I/61+pdx772K...  
3  https://m.media-amazon.com/images/I/41FRyxOo7e...  
4  https://m.media-amazon.com/images/I/61n6YArXF2...   [0.55085355 0.5320103  0.5204488  0.5203953  0.51483893]
Image-only recommendations:
                                       product_title  \
0  Sephora Favorites Beauty Vault: 12 Days of Mak...   
1  4 pcs Professional Eye Makeup Cosmetic Set Con...   
2     Kapitza x Clinique 7 Piece Gift Set. $77 Value 

## Zero Shot Approach

In [6]:
import pandas as pd

df_queries = pd.read_excel("Amazon_recom_queries.xlsx")

amazon_grouped = df_queries.groupby("Queries").agg({
    "Product_title":       list,
    "Product_description": list,
    "Product_link":        list,
    "Image_link":          list
}).reset_index()

amazon_grouped["Model_rec_titles"]       = None
amazon_grouped["Model_rec_descriptions"] = None
amazon_grouped["Model_rec_links"]        = None
amazon_grouped["Model_rec_scores"]       = None

for i, row in amazon_grouped.iterrows():
    q = row["Queries"]
    img_url = row["Image_link"][0]  
    
    recs, scores = unified_query(input_text=q, input_image_path=img_url, k=5, save_dir=approach_dir)    
    amazon_grouped.at[i, "Model_rec_titles"]       = recs["product_title"].tolist()
    amazon_grouped.at[i, "Model_rec_descriptions"] = recs["product_description"].tolist()
    amazon_grouped.at[i, "Model_rec_links"]        = recs.get("product_link", pd.Series()).tolist()
    amazon_grouped.at[i, "Model_rec_scores"]       = scores.tolist()

In [11]:
amazon_grouped.sample(5)

,Queries,Product_title,Product_description,Product_link,Image_link,Model_rec_titles,Model_rec_descriptions,Model_rec_links,Model_rec_scores,mean_similarity@5
37,"Protein Shaping Styling Mist, Super Hold",[Paul Mitchell Super Clean Light Finishing Hai...,[Product Benefits\tVegan. Paraben Free. Hair T...,[https://www.amazon.com/Paul-Mitchell-Super-Cl...,[https://m.media-amazon.com/images/I/611tW2RwB...,"[PERFUMED STYLING MIST – 50 ml, it's a 10 Mira...","[nan, nan, nan, nan, nan]",[],"[0.6938587427139282, 0.6857174634933472, 0.685...",0.685757
19,Herbal Styling Spritz,"[Hair Perfume Oil for Women (Peach Oolong), Ar...","[Product Benefits\tFrizz Control, Conditioning...",[https://www.amazon.com/Perfume-Protection-Dam...,[https://m.media-amazon.com/images/I/61yynwXnq...,"[GLOW BOOSTER SERUM, Collagen, Retinol & Matri...","[nan, nan, nan, nan, nan]",[],"[0.6593971252441406, 0.6585724949836731, 0.658...",0.657407
49,Working Hat with Button Sweatband,[Satin Lined Scrub Caps with Buttons & Sweatha...,[Fabric typeCotton+satin lined scrub caps Care...,[https://www.amazon.com/Yuest-Satin-Buttons-Sw...,[https://m.media-amazon.com/images/I/71AP-A7Ts...,"[JULY'S SONG boys Summer Sun Protection Hat, H...","[nan, nan, nan, nan, nan]",[],"[0.6557657718658447, 0.6545619964599609, 0.645...",0.643587
41,Short Blonde Anime Cosplay Wig,[Kaneles Mens Blonde Wig Short Wig Natural Hai...,[Color\tSoft Blonde Material\tSynthetic Hair T...,[https://www.amazon.com/gp/aw/d/B0861Y7PNB/?_e...,[https://m.media-amazon.com/images/I/51Alt3GUB...,[HSG Cosplay Short Blonde Synthetic Fiber Wigs...,"[nan, Are you ready to rock your world? This f...",[],"[0.6753273010253906, 0.6546528339385986, 0.648...",0.653363
28,Mousse Relaxed Hair,[Sebastian Professional Mousse Forte | Strong-...,[Product Benefits\tGives hair heat resistant b...,[https://www.amazon.com/Sebastian-Professional...,[https://m.media-amazon.com/images/I/61r9MADIc...,"[Ion Straightening Blow Dry Balm, Humidity Blo...","[nan, nan, nan, nan, nan]",[],"[0.6959571838378906, 0.6942131519317627, 0.687...",0.689528


In [8]:
amazon_grouped["mean_similarity@5"] = amazon_grouped["Model_rec_scores"] \
                                            .apply(lambda scores: np.mean(scores))

In [9]:
overall_mean = amazon_grouped["mean_similarity@5"].mean()
overall_mean

np.float64(0.6724964230060577)

In [ ]:
amazon_grouped.to_csv(os.path.join(approach_dir, "model_recommendations.csv"))

In [ ]:
### Fine tuned Model

In [27]:
## SAMPLE TESTING
approach_dir = "artifacts_lora/"
q_text = "photo finish Professional airbrush makeup"
q_img  = "https://temptupro.com/cdn/shop/products/s-one-essential-airbrush-kit-hero_2.jpg?v=1743181132&width=1780"

recs_text, scores_text = unified_query(input_text=q_text,input_image_path=None,save_dir=approach_dir,k=5)
print("Text-only recommendations:")
print(recs_text[["product_title","product_image_url"]], scores_text)

recs_img, scores_img = unified_query(input_text=None,input_image_path=q_img,save_dir=approach_dir,k=5)
print("Image-only recommendations:")
print(recs_img[["product_title","product_image_url"]], scores_img)

recs_both, scores_both = unified_query(input_text=q_text,input_image_path=q_img,save_dir=approach_dir,k=5)
print("Text+Image recommendations:")
print(recs_both[["product_title","product_image_url"]], scores_both)

Text-only recommendations:
                                       product_title  \
0  Svanslashes Eyelash Extensions D Curl - Premiu...   
1  Silicone Lotion Bottles Squeezable Leak Proof ...   
2      Uppercut Deluxe Men's Conditioner - Pack of 3   
3  NOKMOPO Women Fashion Keep Warm Knitting Headb...   
4  613 Blonde Deep Wave 3 Bundles 100% Brazilian ...   

                                   product_image_url  
0  https://m.media-amazon.com/images/I/81TEjbcXTj...  
1  https://m.media-amazon.com/images/I/41+SopylN9...  
2  https://m.media-amazon.com/images/I/61YDP21AdP...  
3  https://m.media-amazon.com/images/I/61-zjZBMXL...  
4  https://m.media-amazon.com/images/I/71ZGWHAjlR...   [0.5009656  0.50083387 0.4996715  0.49935097 0.4992208 ]
Image-only recommendations:
                                       product_title  \
0  Detangling Brush Pink - Detangle Brush - No Ta...   
1  RORASA Magnetic Eyelashes with Magnetic Eyelin...   
2  L.O.L Surprise! Townley Girl Jumbo Hair Access... 

In [28]:
amazon_grouped = df_queries.groupby("Queries").agg({
    "Product_title":       list,
    "Product_description": list,
    "Product_link":        list,
    "Image_link":          list
}).reset_index()

amazon_grouped["Model_rec_titles"]       = None
amazon_grouped["Model_rec_descriptions"] = None
amazon_grouped["Model_rec_links"]        = None
amazon_grouped["Model_rec_scores"]       = None

for i, row in amazon_grouped.iterrows():
    q = row["Queries"]
    img_url = row["Image_link"][0] 
    
    recs, scores = unified_query(input_text=q, input_image_path=img_url, k=5, save_dir=approach_dir)    
    amazon_grouped.at[i, "Model_rec_titles"]       = recs["product_title"].tolist()
    amazon_grouped.at[i, "Model_rec_descriptions"] = recs["product_description"].tolist()
    amazon_grouped.at[i, "Model_rec_links"]        = recs.get("product_link", pd.Series()).tolist()
    amazon_grouped.at[i, "Model_rec_scores"]       = scores.tolist()

In [29]:
amazon_grouped.sample(5)

,Queries,Product_title,Product_description,Product_link,Image_link,Model_rec_titles,Model_rec_descriptions,Model_rec_links,Model_rec_scores
12,EDC Spray and After Shave gift pack,"[Liz Claiborne Curve Black 4.2oz EDC Spray, 3....",[Brand\tLiz Claiborne Item Form\tSpray Item Vo...,[https://www.amazon.com/LIZ-CLAIBORNE-Curve-Sh...,[https://m.media-amazon.com/images/I/61abQC1Mg...,[FU SHEN Brazilian Virgin Human Hair Body Wave...,"[nan, nan, nan, nan, nan]",[],"[0.756136417388916, 0.7554812431335449, 0.7491..."
0,10A Straight Human Hair Bundles with Closure,[10A Straight Bundles with Closure 20 22 24 wi...,[Brand\tWowqueen Beauty Color\tBundles with Cl...,[https://www.amazon.com/Straight-Bundles-Closu...,[https://m.media-amazon.com/images/I/81NdEQli2...,"[Unny Club Triple Shadow (Lovely Pink), Hair S...","[nan, nan, A color-corrector that helps neutra...",[],"[0.9603068828582764, 0.960051417350769, 0.9599..."
13,Eco cotton pads for Face&Neck,[Organic Cotton Rounds Makeup Remover Pads (30...,[Size\t300 Count Brand\tBeautiful Mind Materia...,[https://www.amazon.com/Beautiful-Mind-Cotton-...,[https://m.media-amazon.com/images/I/61cMGUZUs...,[04 COLOURPOP No Filter Matte Concealer 4 g / ...,"[Instantly conceal, cover, correct and contour...",[],"[0.7811999320983887, 0.7677441835403442, 0.755..."
3,Amla Hair Oil,[Dabur Amla Hair Oil - Nourishing Indian Oil f...,[Brand\tDabur Hair Type\tAll Item Weight\t300 ...,[https://www.amazon.com/Dabur-Amla-Hair-Oil-30...,[https://m.media-amazon.com/images/I/71ZKLI829...,[3pieces/pack Synthetic Marlybob Kinky Curl (3...,"[nan, nan, nan, nan, nan]",[],"[0.780077338218689, 0.7764216661453247, 0.7763..."
8,Breathable Nipple Cover,"[Breathable Nipple Covers – No-Stick, Reusable...",[Care instructionsWaterproof OriginImported Co...,[https://www.amazon.com/Bodivault-Breathable-N...,[https://m.media-amazon.com/images/I/519ak7AhI...,[TAN MUJIANG Hair Brush Boar Bristle Brush for...,"[nan, nan, nan, nan, nan]",[],"[0.6637303829193115, 0.6592069864273071, 0.656..."


In [30]:
amazon_grouped["mean_similarity@5"] = amazon_grouped["Model_rec_scores"] \
                                            .apply(lambda scores: np.mean(scores))
overall_mean = amazon_grouped["mean_similarity@5"].mean()
print(overall_mean)
amazon_grouped.to_csv(os.path.join(approach_dir, "model_recommendations.csv"))

0.7937526664733886


## Fine tuned with LORA and optimized with mixed-precision

In [32]:
## SAMPLE TESTING
approach_dir = "artifacts_20k_clip_mp/"
q_text = "photo finish Professional airbrush makeup"
q_img  = "https://temptupro.com/cdn/shop/products/s-one-essential-airbrush-kit-hero_2.jpg?v=1743181132&width=1780"

recs_text, scores_text = unified_query(input_text=q_text,input_image_path=None,save_dir=approach_dir,k=5)
print("Text-only recommendations:")
print(recs_text[["product_title","product_image_url"]], scores_text)

recs_img, scores_img = unified_query(input_text=None,input_image_path=q_img,save_dir=approach_dir,k=5)
print("Image-only recommendations:")
print(recs_img[["product_title","product_image_url"]], scores_img)

recs_both, scores_both = unified_query(input_text=q_text,input_image_path=q_img,save_dir=approach_dir,k=5)
print("Text+Image recommendations:")
print(recs_both[["product_title","product_image_url"]], scores_both)

Text-only recommendations:
                                       product_title  \
0                   human hair bundles (14 16 18, n)   
1  100 Pieces Hair Clips Claw Butterfly Hair Clip...   
2  Retinol Cream 2.5%, Organic with Hyaluronic Ac...   
3  Brazilian Virgin Hair Straight Bundles 8A Grad...   
4  2006 FIESTA SUN BANANABERRY TINI TANNING LOTIO...   

                                   product_image_url  
0  https://m.media-amazon.com/images/I/61IOoIsY2O...  
1  https://m.media-amazon.com/images/I/61WdiMyFR-...  
2  https://m.media-amazon.com/images/I/71edyx7OSj...  
3  https://m.media-amazon.com/images/I/618wpCWZeq...  
4  https://m.media-amazon.com/images/I/31WF0CT7NS...   [0.52508104 0.5152672  0.48301154 0.48092264 0.47569746]
Image-only recommendations:
                                       product_title  \
0       Pure Paws Ultra Silk Cream Conditioner- 8 oz   
1  Organic Doctor Organic Tea Tree Conditioner, 9...   
2  ss Kama Sutra Treasures of The Sea Bath Salts ... 

In [33]:
df_queries = pd.read_excel("Amazon_recom_queries.xlsx")

# Group by unique Queries, collecting Amazon’s ground-truth lists
amazon_grouped = df_queries.groupby("Queries").agg({
    "Product_title":       list,
    "Product_description": list,
    "Product_link":        list,
    "Image_link":          list
}).reset_index()

amazon_grouped["Model_rec_titles"]       = None
amazon_grouped["Model_rec_descriptions"] = None
amazon_grouped["Model_rec_links"]        = None
amazon_grouped["Model_rec_scores"]       = None

# For each unique query, run unified_query and store the top-K recs + scores
for i, row in amazon_grouped.iterrows():
    q = row["Queries"]
    img_url = row["Image_link"][0]  # use the first image for that query
    
    recs, scores = unified_query(input_text=q, input_image_path=img_url, k=5, save_dir=approach_dir)
    
    amazon_grouped.at[i, "Model_rec_titles"]       = recs["product_title"].tolist()
    amazon_grouped.at[i, "Model_rec_descriptions"] = recs["product_description"].tolist()
    amazon_grouped.at[i, "Model_rec_links"]        = recs.get("product_link", pd.Series()).tolist()
    amazon_grouped.at[i, "Model_rec_scores"]       = scores.tolist()



In [34]:
amazon_grouped["mean_similarity@5"] = amazon_grouped["Model_rec_scores"] \
                                            .apply(lambda scores: np.mean(scores))
overall_mean = amazon_grouped["mean_similarity@5"].mean()
print(overall_mean)
amazon_grouped.to_csv(os.path.join(approach_dir, "model_recommendations.csv"))

0.7254888865947726
